In [1]:
import sys
import os

sys.path.append(os.path.abspath(".."))

In [16]:
import numpy as np
import astropy.units as u
import matplotlib.pyplot as plt
import pickle
import datetime
import platform
import gala
import astropy

from gala.units import galactic
from gala.potential import Hamiltonian
from gala.potential import LogarithmicPotential
from gala.dynamics import PhaseSpacePosition
from gala.dynamics.mockstream import (
    MockStreamGenerator,
    FardalStreamDF
)

from tqdm.notebook import tqdm
import time
from mpl_toolkits.mplot3d import Axes3D

from modules.generate_streams import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 00. Test :

In [6]:
# -------------------------------------------------
# short warm-up time configuration
# -------------------------------------------------

time_cfg = TimeConfig(
    t_backward=4 * u.Gyr,
    dt=1 * u.Myr
)

# -------------------------------------------------
# minimal experiment config (single halo, single mass)
# -------------------------------------------------

experiment_cfg = ExperimentConfig(
    time=time_cfg,
    progenitor_masses=[1e4 * u.Msun],   # single mass
    halo_shapes={"spherical": 1.0},     # single halo
    n_particles=100                    # keep small for warm-up
)

# -------------------------------------------------
# Initialize simulator
# -------------------------------------------------

sim = StellarStreamSimulator(experiment_cfg)

# -------------------------------------------------
# Run warm-up
# -------------------------------------------------

results = sim.run()

print("Warm-up complete.")
print(f"Generated {len(results)} stream(s).")

2026-02-20 18:27:08,901 | INFO | StellarStreamSimulator initialized.
2026-02-20 18:27:08,902 | INFO | Starting stream generation experiment.
2026-02-20 18:27:08,904 | INFO | Creating halo: spherical (q=1.0)
2026-02-20 18:27:08,913 | INFO | Generating stream | halo=spherical | M=1.0e+04 Msun
2026-02-20 18:28:08,831 | INFO | Experiment completed.


Warm-up complete.
Generated 1 stream(s).


In [7]:
results

[{'halo': 'spherical',
  'q': 1.0,
  'progenitor_id': 0,
  'mass': <Quantity 10000. solMass>,
  'stream': (<MockStream cartesian, dim=3, shape=(800000,)>,
   <PhaseSpacePosition cartesian, dim=3, shape=(1,)>),
  'orbit': <Orbit cartesian, dim=3, shape=(4000,)>,
  't': <Quantity [ 0.000e+00, -1.000e+00, -2.000e+00, ..., -3.997e+03, -3.998e+03,
             -3.999e+03] Myr>}]

## 01. Generate Streams :

In [8]:
halo_shapes={
    "spherical": 1.0,
    "oblate": 0.8,
    "prolate": 1.2
            }

progenitor_masses=[
    8.0e+03 * u.Msun,
    1.0e+04 * u.Msun,
    2.0e+04 * u.Msun,  
]

time_cfg = TimeConfig(
    t_backward=4 * u.Gyr,
    dt=1 * u.Myr
)

In [9]:
experiment_cfg = ExperimentConfig(
    time=time_cfg,
    progenitor_masses=progenitor_masses,   
    halo_shapes=halo_shapes,     
    n_particles=1500                    
)

sim = StellarStreamSimulator(experiment_cfg)

streams = sim.run()

print("Streams generation complete.")
print(f"Generated {len(streams)} stream(s).")

2026-02-20 18:43:45,619 | INFO | StellarStreamSimulator initialized.
2026-02-20 18:43:45,620 | INFO | Starting stream generation experiment.
2026-02-20 18:43:45,621 | INFO | Creating halo: spherical (q=1.0)
2026-02-20 18:43:45,626 | INFO | Generating stream | halo=spherical | M=8.0e+03 Msun
2026-02-20 18:59:35,617 | INFO | Generating stream | halo=spherical | M=1.0e+04 Msun
2026-02-20 19:15:26,581 | INFO | Generating stream | halo=spherical | M=2.0e+04 Msun
2026-02-20 19:31:21,389 | INFO | Creating halo: oblate (q=0.8)
2026-02-20 19:31:21,399 | INFO | Generating stream | halo=oblate | M=8.0e+03 Msun
2026-02-20 19:48:49,055 | INFO | Generating stream | halo=oblate | M=1.0e+04 Msun
2026-02-20 20:06:21,713 | INFO | Generating stream | halo=oblate | M=2.0e+04 Msun
2026-02-20 20:23:54,951 | INFO | Creating halo: prolate (q=1.2)
2026-02-20 20:23:54,959 | INFO | Generating stream | halo=prolate | M=8.0e+03 Msun
2026-02-20 20:38:40,775 | INFO | Generating stream | halo=prolate | M=1.0e+04 Msun

Streams generation complete.
Generated 1 stream(s).


In [10]:
streams

[{'halo': 'spherical',
  'q': 1.0,
  'progenitor_id': 0,
  'mass': <Quantity 8000. solMass>,
  'stream': (<MockStream cartesian, dim=3, shape=(12000000,)>,
   <PhaseSpacePosition cartesian, dim=3, shape=(1,)>),
  'orbit': <Orbit cartesian, dim=3, shape=(4000,)>,
  't': <Quantity [ 0.000e+00, -1.000e+00, -2.000e+00, ..., -3.997e+03, -3.998e+03,
             -3.999e+03] Myr>},
 {'halo': 'spherical',
  'q': 1.0,
  'progenitor_id': 1,
  'mass': <Quantity 10000. solMass>,
  'stream': (<MockStream cartesian, dim=3, shape=(12000000,)>,
   <PhaseSpacePosition cartesian, dim=3, shape=(1,)>),
  'orbit': <Orbit cartesian, dim=3, shape=(4000,)>,
  't': <Quantity [ 0.000e+00, -1.000e+00, -2.000e+00, ..., -3.997e+03, -3.998e+03,
             -3.999e+03] Myr>},
 {'halo': 'spherical',
  'q': 1.0,
  'progenitor_id': 2,
  'mass': <Quantity 20000. solMass>,
  'stream': (<MockStream cartesian, dim=3, shape=(12000000,)>,
   <PhaseSpacePosition cartesian, dim=3, shape=(1,)>),
  'orbit': <Orbit cartesian, di

In [17]:
#sim.save("../data/gc_stream_ensemble.pkl")

2026-02-21 15:23:44,071 | INFO | Saved 9 streams to ../data/gc_stream_ensemble.pkl
